<a href="https://colab.research.google.com/github/bhkwon89/cp2/blob/main/cp2_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
situation_code = {
    1: '가족관계',
    2: '학업 및 진로',
    3: '학교폭력/따돌림',
    4: '대인관계',
    5: '연애,결혼,출산',
    6: '진로,취업,직장',
    7: '대인관계(부부,자녀)',
    8: '은퇴, 노후준비',
    9: '건강',
    10: '직작,업무,스트레스',
    11: '건강,죽음',
    12: '대인관계(노년)',
    13: '재정' 
}


emotion_code = {
    10: '분노',
    11: '툴툴대는',
    12: '좌절한',
    13: '짜증내는',
    14: '방어적인',
    15: '악의적인',
    16: '안달하는',
    17: '구역질나는',
    18: '노여워하는',
    19: '성가신',
    20: '슬픔',
    21: '실망한',
    22: '비통한',
    23: '후회되는',
    24: '우울한',
    25: '마비된',
    26: '염세적인',
    27: '눈물이 나는',
    28: '낙담한',
    29: '환멸을 느끼는',
    30: '불안',
    31: '두려운',
    32: '스트레스 받는',
    33: '취약한',
    34: '혼란스러운',
    35: '당혹스러운',
    36: '회의적인',
    37: '걱정스러운',
    38: '조심스러운',
    39: '초조한',
    40: '상처',
    41: '질투',
    42: '배신당한',
    43: '고립된',
    44: '충격 받은',
    45: '가난한, 불우한',
    46: '희생된',
    47: '억울한',
    48: '괴로워하는',
    49: '버려진',
    50: '당황',
    51: '고립된(당황한)',
    52: '남의 시선을 의식하는',
    53: '외로운',
    54: '열등감',
    55: '죄책감의',
    56: '부끄러운',
    57: '혐오스러운',
    58: '한심한',
    59: '혼란스러운',
    60: '기쁨',
    61: '감사하는',
    62: '신뢰하는',
    63: '편안한',
    64: '만족스러운',
    65: '흥분',
    66: '느긋',
    67: '안도',
    68: '신이 난',
    69: '자신하는'
}


In [3]:
!pip install gluonnlp pandas tqdm   
!pip install mxnet
!pip install sentencepiece
!pip install transformers
!pip install torch



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-p32u8kdn/kobert-tokenizer_f0c970be4d9d4b61903f1e4b4ca7a839
  Running command git clone -q https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-p32u8kdn/kobert-tokenizer_f0c970be4d9d4b61903f1e4b4ca7a839


In [5]:
pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-jnt_pfbq
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-jnt_pfbq


In [6]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

from kobert import get_tokenizer
from kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from tqdm import tqdm, tqdm_notebook
from sklearn.model_selection import train_test_split

device = torch.device("cuda:0")

In [7]:
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel


tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')
tok = tokenizer.tokenize

# Setting parameters
max_len = 256
batch_size = 16
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [8]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer,vocab, max_len,
                 pad, pair):
   
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))
         
    def __len__(self):
        return (len(self.labels))

In [9]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=6,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [10]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
def predict(sentence):
    dataset = [[sentence, '0']]
    test = BERTDataset(dataset, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(test, batch_size=batch_size, num_workers=2)
    model.eval()
    answer = 0
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        for logits in out:
            logits = logits.detach().cpu().numpy()
            answer = np.argmax(logits)
    return answer

In [102]:
train_set = pd.read_excel('/content/drive/MyDrive/cp2/dataset/감성대화말뭉치(최종데이터)_Training.xlsx')
validation_set = pd.read_excel('/content/drive/MyDrive/cp2/dataset/감성대화말뭉치(최종데이터)_Validation.xlsx')

In [103]:
train_set = train_set.fillna(' ')
validation_set = validation_set.fillna(' ')

In [104]:
# def make_data(df):
#   df_result = pd.DataFrame()
#   for i in range(len(df)):
#     text1 = (df['사람문장1'][i] + ' ' + df['시스템응답1'][i]  + ' ' + 
#                    df['사람문장2'][i]+ ' ' + df['시스템응답2'][i] + ' ' + 
#                    df['사람문장3'][i] + ' ' + df['시스템응답3'][i])
    
#     text2 = (df['사람문장2'][i] + ' ' + df['시스템응답2'][i] + ' ' + 
#                    df['사람문장3'][i] + ' ' + df['시스템응답3'][i] + ' ' +
#                    df['사람문장4'][i] + ' ' + df['시스템응답4'][i])
    
#     df_result = df_result.append(pd.DataFrame({'감정_대분류': [df['감정_대분류'][i]], '문장': [text1]}))
#     df_result = df_result.append(pd.DataFrame({'감정_대분류': [df['감정_대분류'][i]], '문장': [text2]}))

#   return df_result

In [105]:
# train_set = make_data(train_set)
# train_set.to_csv('/content/drive/MyDrive/cp2/dataset/감성대화말뭉치(최종데이터)_Training2.xlsx')

In [106]:
train_set = pd.read_csv('/content/drive/MyDrive/cp2/dataset/감성대화말뭉치(최종데이터)_Training2.xlsx', index_col=0)

In [107]:
validation_set['감정_대분류'].unique()

array(['분노', '슬픔', '불안', '당황', '상처', '기쁨'], dtype=object)

In [109]:
train_set['감정_대분류'].replace('기쁨 ', '기쁨', inplace=True)
train_set['감정_대분류'].replace('불안 ', '불안', inplace=True)

In [110]:
train_set['감정_대분류'].unique()

array(['기쁨', '불안', '당황', '슬픔', '분노', '상처'], dtype=object)

In [111]:
# train_set['문장'] = (train_set['사람문장1'] + ' ' + train_set['시스템응답1']  + ' ' + 
#                    train_set['사람문장2'] + ' ' + train_set['시스템응답2'] + ' ' + 
#                    train_set['사람문장3'] + ' ' + train_set['시스템응답3'] + 
#                    train_set['사람문장4'] + ' ' + train_set['시스템응답4'])
validation_set['문장'] = (validation_set['사람문장1'] + ' ' + validation_set['시스템응답1']  + ' ' +
                        validation_set['사람문장2'] + ' ' + validation_set['시스템응답2'] + ' ' + 
                        validation_set['사람문장3'] + ' ' + validation_set['시스템응답3'] + ' ' +
                        validation_set['사람문장4'] + ' ' + validation_set['시스템응답4'] )

In [112]:

train_set = train_set.loc[:, ['감정_대분류', '문장']]
validation_set = validation_set.loc[:, ['감정_대분류', '문장']]

train_set.dropna(inplace=True)
validation_set.dropna(inplace=True)
train_set.columns = ['label', 'data']
validation_set.columns = ['label', 'data']

from sklearn import preprocessing
le = preprocessing.LabelEncoder()

train_set['label'] = le.fit_transform(train_set['label'])
validation_set['label'] = le.transform(validation_set['label'])

In [113]:
le.classes_

array(['기쁨', '당황', '분노', '불안', '상처', '슬픔'], dtype=object)

In [114]:
train_set

,label,data
0,0,아내가 드디어 출산하게 되어서 정말 신이 나. 아내분이 출산을 하시는군요. 정말 축...
0,0,아 지금 정말 신이 나. 잘 된 일이네요. 아기가 점점 클게 벌써 기대가 되네. 내...
0,3,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야. 약 종류가 ...
0,3,건강할 때 관리 좀 잘할걸 하는 생각이 들더라고. 현재 상황에서 변화를 주기 위해 ...
0,1,고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워. 고등학교 수...
...,...,...
0,5,내 재능이 남들보다 월등한 거는 아니라면서 취업해서 안정적으로 살았으면 좋겠다고 하...
0,0,이렇게 좋은 운동 시설에서 경로 우대로 운동할 수 있다니 참 행운이야. 운동 시설을...
0,0,덕분에 건강도 챙길 수 있고 너무 좋아. 안마기도 있던데 내일은 운동하고 안마기도 ...
0,3,친구 관계가 너무 힘들어. 베푸는 만큼 돌아오지 않는 것 같아. 많이 서운하셨겠어요...


In [ ]:
train_set_data = [[i, str(j)] for i, j in zip(train_set['data'], train_set['label'])]
validation_set_data = [[i, str(j)] for i, j in zip(validation_set['data'], validation_set['label'])]

train_set_data, test_set_data = train_test_split(train_set_data, test_size = 0.2, random_state=4)

train_set_data = BERTDataset(train_set_data, 0, 1, tok, vocab, max_len, True, False)
test_set_data = BERTDataset(test_set_data, 0, 1, tok, vocab, max_len, True, False)
train_dataloader = torch.utils.data.DataLoader(train_set_data, batch_size=batch_size, num_workers=2)
test_dataloader = torch.utils.data.DataLoader(test_set_data, batch_size=batch_size, num_workers=2)

In [ ]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

In [ ]:
torch.save(model, f'/content/drive/MyDrive/Colab Notebooks/SentimentAnalysisKOBert.pt')
torch.save(model.state_dict(), f'/content/drive/MyDrive/Colab Notebooks/SentimentAnalysisKOBert_StateDict.pt')